In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json
import pickle

In [3]:
df = pd.read_csv('/content/drive/MyDrive/집교 2_Team P/user-track-listen_count_filtered5.csv')

In [4]:
df_embedding = pd.read_pickle('/content/drive/MyDrive/집교 2_Team P/lyrics_Embedding/all_embeddings_combined.pkl')

In [5]:
# Pickle 파일 읽기
with open('/content/drive/MyDrive/집교 2_Team P/lyrics_Embedding/all_embeddings_combined.pkl', 'rb') as file:
    data = pickle.load(file)

# DataFrame으로 변환
df_embedding = pd.DataFrame(data, columns=['embedding', 'track_id'])

# track_id를 정수로 변환 (필요하다면)
df_embedding['track_id'] = df_embedding['track_id'].astype(int)

# 'embedding' 열을 768차원의 각 차원으로 나누기
# df_embedding[['embedding_{}'.format(i) for i in range(768)]] = pd.DataFrame(df_embedding['embedding'].tolist(), index=df_embedding.index)

# 'embedding' 열 삭제
# df_embedding = df_embedding.drop(['embedding'], axis=1)

# DataFrame 확인
print(df_embedding.head())


                                           embedding  track_id
0  [0.012072664, 0.17292306, 0.0061238254, 0.0707...         2
1  [-0.17554894, 0.24209566, 0.4195969, -0.185033...         8
2  [-0.096951924, 0.0034472912, 0.005701333, 0.01...      1524
3  [-0.21775067, 0.244962, 0.24090661, 0.1647732,...      1785
4  [-0.069424234, -0.016805744, 0.21406727, -0.27...      1787


In [6]:
df = pd.merge(df, df_embedding, on=['track_id'], how='left')

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import numpy as np
from sklearn.metrics import mean_squared_error
# 가상의 데이터 프레임 생성 (예시)
# ...

# Label Encoding
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()

df['user_id'] = user_encoder.fit_transform(df['user_id'])
df['track_id'] = item_encoder.fit_transform(df['track_id'])
df.loc[df['listen_count_bin'] == '10~2704', 'listen_count_bin'] = 10
df['listen_count_bin'] = df['listen_count_bin'].astype(int)

# PyTorch DataLoader에 맞게 데이터 변환
def df_to_tensor(dataset):
    users = torch.tensor(dataset['user_id'].values, dtype=torch.long)
    items = torch.tensor(dataset['track_id'].values, dtype=torch.long)
    ratings = torch.tensor(dataset['listen_count_bin'].values, dtype=torch.float)
    return users, items, ratings

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

train_users, train_items, train_ratings = df_to_tensor(train_df)
test_users, test_items, test_ratings = df_to_tensor(test_df)

train_data = TensorDataset(train_users, train_items, train_ratings)
test_data = TensorDataset(test_users, test_items, test_ratings)

train_loader = DataLoader(train_data, batch_size=256, shuffle=True)
test_loader = DataLoader(test_data, batch_size=256, shuffle=False)

class NeuMF(nn.Module):
    def __init__(self, num_users, num_items, embedding_size, mlp_hidden_size):
        super(NeuMF, self).__init__()
        # Matrix Factorization
        self.user_embedding_mf = nn.Embedding(num_users, embedding_size)
        self.item_embedding_mf = nn.Embedding(num_items, embedding_size)
        # Multi-Layer Perceptron
        self.user_embedding_mlp = nn.Embedding(num_users, mlp_hidden_size)
        self.item_embedding_mlp = nn.Embedding(num_items, mlp_hidden_size)
        self.mlp_layers = nn.Sequential(
            nn.Linear(2 * mlp_hidden_size, mlp_hidden_size),
            nn.ReLU(),
            nn.Linear(mlp_hidden_size, int(mlp_hidden_size/2)),
            nn.ReLU(),
            nn.Linear(int(mlp_hidden_size/2), int(mlp_hidden_size/4)),
        )
        # Final Layer
        self.final_layer = nn.Linear(int(mlp_hidden_size/4+embedding_size), 1)

    def forward(self, user, item):
        # Matrix Factorization
        user_embedding_mf = self.user_embedding_mf(user)
        item_embedding_mf = self.item_embedding_mf(item)
        mf_output = torch.mul(user_embedding_mf, item_embedding_mf)

        # Multi-Layer Perceptron
        user_embedding_mlp = self.user_embedding_mlp(user)
        item_embedding_mlp = self.item_embedding_mlp(item)
        mlp_input = torch.cat((user_embedding_mlp, item_embedding_mlp), dim=1)
        mlp_output = self.mlp_layers(mlp_input)
        # Concatenate MF and MLP outputs
        final_input = torch.cat((mf_output, mlp_output), dim=1)

        # Final prediction
        prediction = self.final_layer(final_input)
        return prediction.view(-1)


# NeuMF 모델 초기화
num_users = len(user_encoder.classes_)
num_items = len(item_encoder.classes_)
print("num_users:",num_users)
print("num_items:",num_items)


# 학습 및 평가 코드는 이전과 유사
# ...
# CUDA 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
def train_(emb,mlp,n_epoch):
  embedding_size = emb
  mlp_hidden_size = mlp

  neumf_model = NeuMF(num_users, num_items, embedding_size, mlp_hidden_size)

  # 손실 함수 및 최적화 함수 정의
  criterion = nn.MSELoss()
  optimizer = optim.Adam(neumf_model.parameters(), lr=0.001)
  # NCF 모델 정의 및 GPU로 이동
  # model = NCF(num_users=len(user_encoder.classes_), num_items=len(item_encoder.classes_), embedding_size=embedding_size)
  neumf_model.to(device)
  criterion = nn.MSELoss()
  optimizer = optim.Adam(neumf_model.parameters(), lr=0.001)
  # tqdm을 사용하여 학습 및 테스트 진행 상황 확인
  num_epochs = n_epoch
  for epoch in range(num_epochs):
      neumf_model.train()
      total_loss = 0
      for user, item, rating in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
          optimizer.zero_grad()
          user, item, rating = user.to(device), item.to(device), rating.to(device)  # GPU로 이동
          output = neumf_model(user, item)
          loss = criterion(output, rating.unsqueeze(1))
          loss.backward()
          optimizer.step()
          total_loss += loss.item()

      avg_loss = total_loss / len(train_loader)
      print(f'Epoch {epoch+1}/{num_epochs}, Avg. Loss: {avg_loss:.4f}')

      # 각 에폭이 끝날 때마다 테스트 데이터에 대한 예측 수행
      neumf_model.eval()
      all_predictions = []
      with torch.no_grad():
          for user, item, _ in tqdm(test_loader, desc=f'Testing Epoch {epoch+1}'):
              user, item = user.to(device), item.to(device)  # GPU로 이동
              output = neumf_model(user, item)
              all_predictions.append(output)

      # RMSE 계산
      predictions = torch.cat(all_predictions).squeeze().cpu().numpy()  # CPU로 이동 후 numpy로 변환
      rmse = np.sqrt(mean_squared_error(test_df['listen_count_bin'].values, predictions))
      print(f'Epoch {epoch+1}/{num_epochs}, RMSE on test set: {rmse}')

num_users: 23761
num_items: 28378
Using device: cuda


In [11]:
train_(64,256,10)

Epoch 1/10:   0%|          | 0/14516 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([256, 1])) that is different to the input size (torch.Size([256])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Epoch 1/10: 100%|█████████▉| 14501/14516 [01:37<00:00, 153.78it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([168, 1])) that is different to the input size (torch.Size([168])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Epoch 1/10: 100%|██████████| 14516/14516 [01:38<00:00, 147.82it/s]


Epoch 1/10, Avg. Loss: 1.6497


Testing Epoch 1: 100%|██████████| 3629/3629 [00:12<00:00, 282.12it/s]


Epoch 1/10, RMSE on test set: 1.2815246719344136


Epoch 2/10:   0%|          | 0/14516 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([256, 1])) that is different to the input size (torch.Size([256])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Epoch 2/10: 100%|█████████▉| 14510/14516 [01:39<00:00, 147.11it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([168, 1])) that is different to the input size (torch.Size([168])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Epoch 2/10: 100%|██████████| 14516/14516 [01:39<00:00, 145.53it/s]


...

Epoch 7/10, Avg. Loss: 1.6427


Testing Epoch 7: 100%|██████████| 3629/3629 [00:13<00:00, 278.06it/s]


Epoch 7/10, RMSE on test set: 1.2812932811738698


Epoch 8/10:   0%|          | 0/14516 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([256, 1])) that is different to the input size (torch.Size([256])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Epoch 8/10:  10%|█         | 1490/14516 [00:10<01:29, 145.69it/s]


KeyboardInterrupt: ignored

In [9]:
train_(128,128,10)

Epoch 1/10:   0%|          | 0/14516 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([256, 1])) that is different to the input size (torch.Size([256])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Epoch 1/10: 100%|█████████▉| 14512/14516 [01:34<00:00, 159.57it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([168, 1])) that is different to the input size (torch.Size([168])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Epoch 1/10: 100%|██████████| 14516/14516 [01:34<00:00, 153.57it/s]


Epoch 1/10, Avg. Loss: 1.6548


Testing Epoch 1: 100%|██████████| 3629/3629 [00:12<00:00, 286.96it/s]


Epoch 1/10, RMSE on test set: 1.2814876983654866


Epoch 2/10:   0%|          | 0/14516 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([256, 1])) that is different to the input size (torch.Size([256])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Epoch 2/10: 100%|█████████▉| 14509/14516 [01:36<00:00, 143.04it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([168, 1])) that is different to the input size (torch.Size([168])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Epoch 2/10: 100%|██████████| 14516/14516 [01:37<00:00, 149.53it/s]


...

Epoch 3/10, Avg. Loss: 1.6431


Testing Epoch 3: 100%|██████████| 3629/3629 [00:12<00:00, 284.79it/s]


Epoch 3/10, RMSE on test set: 1.2816247907843161


Epoch 4/10:   0%|          | 0/14516 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([256, 1])) that is different to the input size (torch.Size([256])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Epoch 4/10:  47%|████▋     | 6889/14516 [00:45<00:50, 150.97it/s]


KeyboardInterrupt: ignored

In [ ]:
train_(256,256,10)